# Create a Bucket and Lock it Down to a Specific VPC

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
s3 = boto3.Session().client(service_name="s3", region_name=region)
ec2 = boto3.Session().client(service_name="ec2", region_name=region)

In [ ]:
import time

timestamp = int(time.time())

bucket_secure_name = "bucket-secure-bucket-policy-vpc-{}".format(timestamp)
print(bucket_secure_name)

In [ ]:
!aws s3 mb s3://$bucket_secure_name

In [ ]:
!aws s3 ls s3://$bucket_secure_name

In [ ]:
different_vpc_id = "blah"

In [ ]:
all_vpcs = ec2.describe_vpcs()

vpc_id = all_vpcs["Vpcs"][0]["VpcId"]

print(vpc_id)

In [ ]:
# Create the bucket policy
bucket_policy_deny = {
    "Version": "2008-10-17",
    "Statement": [
        {
            "Effect": "Deny",
            "Principal": "*",
            "Action": ["s3:ListBucket"],
            "Resource": ["arn:aws:s3:::{}".format(bucket_secure_name)],
            "Condition": {
                "StringNotEquals": {
                    #                        "aws:sourceVpc": different_vpc_id
                    "aws:sourceVpc": vpc_id
                }
            },
        }
    ],
}

# Set the Deny Policy on the Bucket

In [ ]:
import json
import time

response = s3.put_bucket_policy(Bucket=bucket_secure_name, Policy=json.dumps(bucket_policy_deny))

print(response)

time.sleep(30)

# Verify `AccessDenied`

In [ ]:
!aws s3 ls s3://$bucket_secure_name

# Remove the Policy

In [ ]:
response = s3.delete_bucket_policy(Bucket=bucket_secure_name)
print(response)

time.sleep(30)

# Verify Access After Removing the Policy

In [ ]:
!aws s3 ls s3://$bucket_secure_name

# Remove the Bucket

In [ ]:
!aws s3 rm s3://$bucket_secure_name

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}